In [14]:
from control.jumpy_control import Jumpy, JumpyHelper
import cv2
from pynput import mouse
from pynput.mouse import Button
from threading import Thread
import time
import json
import numpy as np
import os

In [15]:
include_previous_frame = False
ignore_zero_output = False
frames_ahead = 0
previous_outputs = 0

In [16]:
d='./gameplay_logs'
folders = [ os.path.join(d, name) for name in os.listdir(d) if os.path.isdir(os.path.join(d, name)) ]
folders 

['./gameplay_logs/1553448946.270552',
 './gameplay_logs/1553448965.3654816',
 './gameplay_logs/1553449015.2570503',
 './gameplay_logs/1553449071.8743048']

In [17]:
for folder in folders:
    print("processing folder ",folder,"...")
    
    npz_file_path = f"{folder}/features_X_y.npz"
    if os.path.exists(npz_file_path):
        print("file already exists, skipping...")
        continue
        
    with open(f'{folder}/game_info.json') as f:
        game_info = json.load(f)


    X = None
    y = None

    limit = len(game_info['images'])-45 #Skip last 45 frames (after loss)
    previous_features = None
    keys = list(game_info['images'].keys())
    for i in range(len(keys)):
        image_name = keys[i]
            
        if i>=limit:
            break;
        if i < 10: #skip first 10
            continue

        if frames_ahead:
            ahead_image_name = keys[i+frames_ahead]
            x_drag = np.atleast_2d(game_info["images"][ahead_image_name]["prev_drag_x"])
        else:
            x_drag = np.atleast_2d(game_info["images"][image_name]["prev_drag_x"])
            
        
        if ignore_zero_output and x_drag==0:
            continue
        
        img = cv2.imread(f"{folder}/images/{image_name}")
        helper = JumpyHelper(img)
        features = np.atleast_2d(helper.get_features())
        to_add = features
            
        if include_previous_frame:
            if previous_features is None:
                previous_features = np.zeros_like(features)
            
            to_add = np.append(to_add, previous_features, axis=1) 
        
        if previous_outputs:
            prev_x_drags = [0] * previous_outputs
            j = 0
            while j < previous_outputs and (i-j-1)>=0:
                prev_x_drags[j] = game_info["images"][keys[(i-j-1)]]["prev_drag_x"]
                j+=1
            to_add = np.append(to_add, np.atleast_2d(prev_x_drags), axis=1)
    
        if X is None:
            X = to_add
        else:
            X = np.append(X, to_add, axis=0)
        
        previous_features = features

        if y is None:
            y = x_drag
        else:
            y = np.append(y, x_drag, axis=0)



    with open(npz_file_path, "wb") as f:
        np.savez(f, X=X, y=y)
    
    print("Done with folder "+folder)
    
print("Done generating!")

processing folder  ./gameplay_logs/1553448946.270552 ...
Done with folder ./gameplay_logs/1553448946.270552
processing folder  ./gameplay_logs/1553448965.3654816 ...
Done with folder ./gameplay_logs/1553448965.3654816
processing folder  ./gameplay_logs/1553449015.2570503 ...
Done with folder ./gameplay_logs/1553449015.2570503
processing folder  ./gameplay_logs/1553449071.8743048 ...
Done with folder ./gameplay_logs/1553449071.8743048
Done generating!


In [18]:
print(X[5,-5:], X[6,-5:])
print(y[0:5])
print(X.shape)

[1 0 0 0 0] [1 0 0 0 0]
[[ 8]
 [10]
 [ 6]
 [ 3]
 [ 2]]
(1285, 1278)


In [19]:
X=None
y=None

for folder in folders:
    with open(f"{folder}/features_X_y.npz", "rb") as f:
        data = np.load(f)
        if X is None:
            X = data["X"]
        else:
            X = np.append(X, data["X"], axis=0)
        if y is None:
            y = data["y"]
        else:
            y = np.append(y, data["y"], axis=0)
            

with open(f"{d}/features_X_y.npz", "wb") as f:
    np.savez(f, X=X, y=y)
    
print("done")

done


In [20]:
X.shape

(3677, 1278)

In [21]:
y.shape

(3677, 1)